# Roman Numeral Calculator


In [1]:
import json

## Calculator Class

In [2]:
class Calculator():
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def add(self):
        return self.a + self.b
    
    def sub(self):
        return self.a - self.b
    
    def mul(self):
        return self.a * self.b
    
    def __str__(self):
        return f"A: {self.a}\nB: {self.b}"

## RomanToNumber Class

In [10]:
class RomanToNumber():
    def __init__(self, a):
        self.a = list(a.upper())

        with open("roman.json", 'r') as f:
            self.romans = json.load(f)
        
        with open("substraction.json", "r") as f:
            self.substraction_pairs = json.load(f)
        
    def to_num(self):
        number = 0

        if len(self.a) == 1:
            number = self.romans[self.a[0]]

        elif len(self.a) == 2:
            pair = self.a[0] + self.a[1]
            if pair in self.substraction_pairs["Substraction_rule"]:
                index = self.substraction_pairs["Substraction_rule"].index(pair)
                number = self.substraction_pairs["Substituting_values"][index]
            else: 
                for i in self.a:
                    number += self.romans[i]

        else:
            for i in self.a[:-2]:
                number += self.romans[i]
            pair = self.a[-2] + self.a[-1]
            if pair in self.substraction_pairs["Substraction_rule"]:
                index = self.substraction_pairs["Substraction_rule"].index(pair)
                number += self.substraction_pairs["Substituting_values"][index]
            else:
                number += self.romans[self.a[-2]]
                number += self.romans[self.a[-1]]
        
        return number

    def __str__(self):
        return f"Roman number is: {self.a}"

## NumberToRoman Class

In [ ]:
class NumberToRoman():
    def __init__(self, a):
        self.a = str(a)
    
        with open("substraction.json", "r") as f:
            self.substraction_pairs = json.load(f)
        
        with open("roman.json", 'r') as f:
            self.romans = json.load(f)
        
        with open("unit_numbers.json") as f:
            self.unit_romans = json.load(f)
        
    def add_vinculum(self, string):
        return "".join([char + '\u0305' for char in string])
    
    def to_roman_helper(self, number):
        roman_number = str()
        roman_list = []

        if 1 <= (number // 100) < 3:
            count = number // 100
            for _ in range(count):
                roman_list.append("C")
        
        elif 1 <= (number // 10) < 4:
            count = number // 10
            for _ in range(count):
                roman_list.append("X")
        
        roman_number = ''.join(roman_list)
        return roman_number
    
    def to_roman(self):
        length = len(self.a)
        num_list = list(self.a)
        places_list = []
        roman_list = []

        if length == 1:
            return self.unit_romans[self.a]
        
        for i in num_list:
            places_list.append(int(i) * (10**(length-1)))
            length -= 1
        
        for i in places_list:
            if len(str(i)) >= 3 and i // 1000:
                if 1 < i // 1000 < 4: 
                    for _ in range(i // 1000):
                        roman_list.append("M")
                else:
                    a = i // 1000
                    roman_list.append(self.add_vinculum(self.unit_romans[str(a)]))
            elif len(str(i)) == 3 and i // 500:
                if i // 500:
                    remainder = i - 500
                    if remainder // 100 <= 3:  # Fixed: Changed < 4 to <= 3
                        roman_numeral_remainder = self.to_roman_helper(remainder)
                        roman_list.append("D" + roman_numeral_remainder)
                    else: 
                        roman_list.append("CM")

            elif len(str(i)) == 3 and i // 100:
                if i // 100:
                    remainder = i - 100
                    if remainder // 100 <= 2:  # Fixed: Changed < 3 to <= 2
                        roman_numeral_remainder = self.to_roman_helper(remainder)
                        roman_list.append("C" + roman_numeral_remainder)
                    else: 
                        roman_list.append("CD")

            elif len(str(i)) == 2: 
                if i // 50:
                    remainder = i - 50
                    if remainder // 10 <= 3:  # Fixed: Changed < 4 to <= 3
                        roman_numeral_remainder = self.to_roman_helper(remainder)
                        roman_list.append("L" + roman_numeral_remainder)
                    else:
                        roman_list.append("XC")
                else:
                    remainder = i - 10
                    if remainder // 10 <= 2:  # Fixed: Changed < 3 to <= 2
                        roman_numeral_remainder = self.to_roman_helper(remainder)
                        roman_list.append("X" + roman_numeral_remainder)
                    else:
                        roman_list.append("XL")
            
            elif len(str(i)) == 1 and i != 0:
                roman_list.append(self.unit_romans[str(i)])
        
        roman_number = ''.join(roman_list)
        return roman_number

## Interactive Roman Calculator

In [5]:
# Get user input for Roman numerals
a = RomanToNumber(input("Enter 1st Roman Number: ")).to_num()
b = RomanToNumber(input("Enter 2nd Roman Number: ")).to_num()

# Get operator
operator = input("\nEnter the Operator (+, -, *): ")

# Perform calculation
solution = Calculator(a, b)

if operator == "+":
    result = solution.add()
elif operator == '-':
    result = solution.sub()
elif operator == '*':
    result = solution.mul()
else:
    print("Use only *, -, +")
    result = None

# Convert result back to Roman numeral
if result is not None:
    converter = NumberToRoman(result)
    print(f"\nResult: {converter.to_roman()}")


Result: XXII


## Example Test Cases

### Example 1: Addition (XIV + VI)

In [6]:
# Example 1: XIV (14) + VI (6) = XX (20)
num1 = RomanToNumber("XIV").to_num()
num2 = RomanToNumber("VI").to_num()
calc = Calculator(num1, num2)
result = calc.add()
roman_result = NumberToRoman(result).to_roman()
print(f"XIV + VI = {roman_result} ({num1} + {num2} = {result})")

XIV + VI = XX (14 + 6 = 20)


### Example 2: Subtraction (L - XX)

In [7]:
# Example 2: L (50) - XX (20) = XXX (30)
num1 = RomanToNumber("L").to_num()
num2 = RomanToNumber("XX").to_num()
calc = Calculator(num1, num2)
result = calc.sub()
roman_result = NumberToRoman(result).to_roman()
print(f"L - XX = {roman_result} ({num1} - {num2} = {result})")

L - XX = XXX (50 - 20 = 30)


### Example 3: Multiplication (XII * III)

In [8]:
# Example 3: XII (12) * III (3) = XXXVI (36)
num1 = RomanToNumber("XII").to_num()
num2 = RomanToNumber("III").to_num()
calc = Calculator(num1, num2)
result = calc.mul()
roman_result = NumberToRoman(result).to_roman()
print(f"XII * III = {roman_result} ({num1} * {num2} = {result})")

XII * III = XXXVI (12 * 3 = 36)


### Example 4: Large Number Conversion

In [ ]:
# Example 4: Converting 3888 to Roman numerals
num = 3888
roman = NumberToRoman(num).to_roman()
print(f"{num} in Roman numerals: {roman}")

# Verify by converting back
decimal = RomanToNumber(roman).to_num()
print(f"{roman} back to decimal: {decimal}")
print(f"Conversion successful: {decimal == num}")

3888 in Roman numerals: MMMDLXXXVIII
MMMDLXXXVIII back to decimal: 3588
